In [18]:
import mediapipe as mp
import pickle 
import numpy as np
import cv2
import pandas as pd

mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic

In [15]:
# Load model
with open('./body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [16]:
model

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('svc', SVC(probability=True))])

In [17]:
hand_keypoints = 21 * 3
pose_keypoints = 15 * 3
hand_empty_keypoints = np.empty(hand_keypoints)
pose_empty_keypoints = np.empty(pose_keypoints)
hand_empty_keypoints[:] = 0.0 #np.nan
pose_empty_keypoints[:] = 0.0 #np.nan
hand_empty_keypoints = list(hand_empty_keypoints)
pose_empty_keypoints = list(pose_empty_keypoints)

In [19]:
cap = cv2.VideoCapture('salud.mp4')
threshold = 0.7
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            # Resize frame
            #frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation = cv2.INTER_AREA)
            # Change color from BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame.flags.writeable = False
            # Detect landmarks
            results = holistic.process(frame)
            
             # Mano izquierda (rojo)
            mp_drawing.draw_landmarks(
                frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=1),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2))

           # Mano derecha (verde)
            mp_drawing.draw_landmarks(
                frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1),
                mp_drawing.DrawingSpec(color=(57, 143, 0), thickness=2))

           # Postura
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(128, 0, 255), thickness=2, circle_radius=1),
                mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))
            
            try:
                
                # Add coords to list row
                row = []            
                ## Get rhand landmarks
                if results.right_hand_landmarks:
                    for landmark in results.right_hand_landmarks.landmark:
                        row.append(landmark.x)
                        row.append(landmark.y)
                        row.append(landmark.z)
                else:
                    row.extend(hand_empty_keypoints)
                ## Get lhand landmarks
                if results.left_hand_landmarks:
                    for landmark in results.left_hand_landmarks.landmark:
                        row.append(landmark.x)
                        row.append(landmark.y)
                        row.append(landmark.z)
                else:
                    row.extend(hand_empty_keypoints)
                    
                ## Get pose landmarks
                if results.pose_landmarks:
                    count = 0
                    for landmark in results.pose_landmarks.landmark:
                        row.append(landmark.x)
                        row.append(landmark.y)
                        row.append(landmark.z)
                        count += 1
                        if count == 15:
                            break
                else:
                    row.extend(pose_empty_keypoints)
            except:
                pass 
           
            if len(row)==171:
                X = pd.DataFrame([row])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
                output_class = round(body_language_prob[np.argmax(body_language_prob)], 2)
                
                if output_class > threshold:
                    print(f"{body_language_class}: {output_class}")
                    # RECTAGULO SUPERIOR IZQUIERDA
                    cv2.rectangle(frame, (0,0), (250, 60), (245, 117, 16),2)

                    # Display Class
                    cv2.putText(frame, 'CLASS'  # texto enunciado
                                , (110,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(frame, body_language_class.split(' ')[0]   #texto valor class
                                , (110,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                    # Display Probability
                    cv2.putText(frame, 'PROB'
                                , (15,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(frame, str(output_class)
                                , (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
            frame = cv2.resize(frame, (600, 400))
            cv2.imshow("Imagen a detectar", frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        else:
            break
cap.release()
cv2.destroyAllWindows()

dificultad_respirar: 0.8
dificultad_respirar: 0.75
dificultad_respirar: 0.72
dificultad_respirar: 0.74
dificultad_respirar: 0.73
dificultad_respirar: 0.71
dificultad_respirar: 0.72
dificultad_respirar: 0.71
dificultad_respirar: 0.72
dificultad_respirar: 0.74
